# Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import string
from tqdm import tqdm
tqdm.pandas()
from collections import Counter
from itertools import tee
import re

from IPython.display import display

import numpy as np
from scipy import stats
from scipy.sparse import hstack as sparse_hstack
import pandas as pd

import matplotlib.pyplot as plt; plt.rcParams['figure.dpi'] = 100
import matplotlib.gridspec as gridspec
import seaborn as sns; sns.set()
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [ ]:
!pip install eli5
import eli5

!pip install pycountry
import pycountry

!pip install contractions
import contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 5.1 MB/s 
     |████████████████████████████████| 133 kB 41.1 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=0c0f8f9f925dfd9427d857fe090c648ccf7e9bd09b165c4334bb2acd0cf7ad3b
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 5.7.16 requires jinja2<=3.0.0, but you have jinja2 3.1.2 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompa

# Data Preparation

In [ ]:
data=pd.read_csv("data_39.csv", engine='python', on_bad_lines='warn')
data.head(1)

,title,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,...,company_profile_length,requirements_length,benefits_length,description_length,country,state,city,min_salary,max_salary,salary_specified
0,marketing intern,Marketing,food52 create groundbreaking award win cooking...,food52 fast grow james beard award win online ...,experience content management system major plu...,NaN,0,1,0,Other,...,90,77,0,84,US,NY,New York,0,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   17880 non-null  object
 1   department              6333 non-null   object
 2   company_profile         14572 non-null  object
 3   description             17878 non-null  object
 4   requirements            15178 non-null  object
 5   benefits                10657 non-null  object
 6   telecommuting           17880 non-null  int64 
 7   has_company_logo        17880 non-null  int64 
 8   has_questions           17880 non-null  int64 
 9   employment_type         14409 non-null  object
 10  required_experience     10830 non-null  object
 11  required_education      9775 non-null   object
 12  industry                12977 non-null  object
 13  function                11425 non-null  object
 14  fraudulent              17880 non-null  int64 
 15  co

In [ ]:
# dropped combine text cause for logistic regression is not needed

text_features = ['title', 'company_profile', 'description', 'requirements', 'benefits']
bin_features = ['telecommuting', 'has_company_logo', 'has_questions', 'has_dollar_sign', 'has_email', 'has_phone_number', 'has_company_profile', 
                'has_requirements', 'has_benefits', 'has_employment_type', 'has_description', 'salary_specified']
cat_features = ['department', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'country', 'state', 'city']

num_features = ['company_profile_length', 'requirements_length', 'benefits_length', 'description_length',
                'min_salary', 'max_salary', 'combined_text_length']

#data.drop('job_id', axis=1, inplace=True)
data.drop('combined_text', axis=1, inplace=True)
#data.drop('combined_text_length', axis=1, inplace=True)

data.head()

,title,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,...,company_profile_length,requirements_length,benefits_length,description_length,country,state,city,min_salary,max_salary,salary_specified
0,marketing intern,Marketing,food52 create groundbreaking award win cooking...,food52 fast grow james beard award win online ...,experience content management system major plu...,NaN,0,1,0,Other,...,90,77,0,84,US,NY,New York,0,0,0
1,customer service cloud video production,Success,90 second world cloud video production service...,organise focus vibrant awesome passion custome...,expect key responsibility communicate client 9...,get usthrough part 90 second team gain experie...,0,1,0,Full-time,...,122,123,117,208,NZ,Unspecified,Auckland,0,0,0
2,commission machinery assistant cma,NaN,valor service provide workforce solution meet ...,client locate houston actively seek experience...,implement pre commission commission procedure ...,NaN,0,1,0,NaN,...,79,123,0,30,US,IA,Wever,0,0,0
3,account executive washington dc,Sales,passion improve quality life geography heart e...,company esri environmental system research ins...,education bachelor master gi business administ...,culture anything corporate collaborative creat...,0,1,0,Full-time,...,58,124,79,226,US,DC,Washington,0,0,0
4,bill review manager,NaN,spotsource solution llc global human capital m...,job title itemization review managerlocation f...,qualification right license state texasdiploma...,full benefit offer,0,1,1,Full-time,...,155,66,3,131,US,FL,Fort Worth,0,0,0


In [ ]:
# fill na values in text columns with empty string

for feature_name in text_features[1:]:
    data[feature_name].fillna('', inplace=True)

In [ ]:
# Fill the rest of the columns null value with 'na'

data.fillna('na', inplace=True)

In [ ]:
# Check to see if all values have been filled up

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   17880 non-null  object
 1   department              17880 non-null  object
 2   company_profile         17880 non-null  object
 3   description             17880 non-null  object
 4   requirements            17880 non-null  object
 5   benefits                17880 non-null  object
 6   telecommuting           17880 non-null  int64 
 7   has_company_logo        17880 non-null  int64 
 8   has_questions           17880 non-null  int64 
 9   employment_type         17880 non-null  object
 10  required_experience     17880 non-null  object
 11  required_education      17880 non-null  object
 12  industry                17880 non-null  object
 13  function                17880 non-null  object
 14  fraudulent              17880 non-null  int64 
 15  ha

# Logistic Regression 

In [ ]:
# Formulate stratified cross validation splitter 

skf = StratifiedKFold(n_splits=4, random_state=42, shuffle = True)

In [ ]:
# Segment data properly to pass into logistic regression

X, y = data.drop('fraudulent', axis=1), data.fraudulent

In [ ]:
# Create the column transformers

num_transformer = Pipeline(steps=[('scaler', StandardScaler())])
cat_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
text_transformer = Pipeline(steps=[('tfidf', TfidfVectorizer(ngram_range=(1, 2)))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features),
        *[(feature_name, text_transformer, feature_name) 
          for feature_name in text_features]
    ]
)

In [ ]:
# Create a pipeline for preprocessing before using the model

log_reg_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', LogisticRegression())])

In [ ]:
# Pre-processing will only be applied to training of each split due to the pipeline passed and the nature of cross_validate function.
# Time will be recorded

%%time
cv_scores = cross_validate(log_reg_pipe, X, y, return_train_score=True, cv=skf, 
                           scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'neg_mean_squared_error'], n_jobs=-1)

print(f'Accuracy on train part: {cv_scores["train_accuracy"]}, mean: {cv_scores["train_accuracy"].mean()}')
print(f'Accuracy on test part:  {cv_scores["test_accuracy"]}, mean: {cv_scores["test_accuracy"].mean()}')

print(f'Precision on train part: {cv_scores["train_precision"]}, mean: {cv_scores["train_precision"].mean()}')
print(f'Precision on test part:  {cv_scores["test_precision"]}, mean: {cv_scores["test_precision"].mean()}')

print(f'Recall on train part: {cv_scores["train_recall"]}, mean: {cv_scores["train_recall"].mean()}')
print(f'Recall on test part:  {cv_scores["test_recall"]}, mean: {cv_scores["test_recall"].mean()}')

print(f'F1 on train part: {cv_scores["train_f1"]}, mean: {cv_scores["train_f1"].mean()}')
print(f'F1 on test part:  {cv_scores["test_f1"]}, mean: {cv_scores["test_f1"].mean()}')

print(f'ROC AUC on train part: {cv_scores["train_roc_auc"]}, mean: {cv_scores["train_roc_auc"].mean()}')
print(f'ROC AUC on test part:  {cv_scores["test_roc_auc"]}, mean: {cv_scores["test_roc_auc"].mean()}')

print(f'mse on train part: {cv_scores["train_neg_mean_squared_error"]}, mean: {cv_scores["train_neg_mean_squared_error"].mean()}')
print(f'mse on test part:  {cv_scores["test_neg_mean_squared_error"]}, mean: {cv_scores["test_neg_mean_squared_error"].mean()}')

Accuracy on train part: [0.99545116 0.99552573 0.99492916 0.99492916], mean: 0.9952087994034303
Accuracy on test part:  [0.98568233 0.98389262 0.98389262 0.97964206], mean: 0.9832774049217001
Precision on train part: [0.99495798 0.996633   0.9965812  0.99828473], mean: 0.9966142276353153
Precision on test part:  [0.99350649 0.97368421 0.97385621 0.95      ], mean: 0.9727617282957841
Recall on train part: [0.91076923 0.91076923 0.89830508 0.89676425], mean: 0.90415194974517
Recall on test part:  [0.70833333 0.68518519 0.68663594 0.61290323], mean: 0.6732644222563577
F1 on train part: [0.95100402 0.95176849 0.94489465 0.94480519], mean: 0.9481180877887851
F1 on test part:  [0.82702703 0.80434783 0.80540541 0.74509804], mean: 0.7954695744337688
ROC AUC on train part: [0.99989945 0.99991717 0.99992164 0.9999133 ], mean: 0.9999128888645683
ROC AUC on test part:  [0.98812338 0.98459402 0.99139669 0.98891972], mean: 0.988258453544654
mse on train part: [-0.00454884 -0.00447427 -0.00507084 -0.

In [ ]:
### Check model's weights, let's first transform them first

%%time
feature_names = num_features.copy()

num_features_scaled = StandardScaler().fit_transform(data[num_features])
X = num_features_scaled

feature_names += bin_features
X = np.hstack([X, data[bin_features]])


for feature_name in cat_features:
    encoder = OneHotEncoder()
    encoded_feature = encoder.fit_transform(data[feature_name].values.reshape(-1, 1))
    
    X = sparse_hstack([X, encoded_feature])
    f_names = list(map(lambda cat: f'{feature_name}:{cat}', encoder.categories_[0]))
    feature_names += f_names

for feature_name in text_features:
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    vectorized_feature = vectorizer.fit_transform(data[feature_name])
    
    X = sparse_hstack([X, vectorized_feature])
    sorted_phrases = [pair[0] for pair in list(sorted(vectorizer.vocabulary_.items(), 
                                                      key=lambda pair: pair[1]))]
    f_names = list(map(lambda phrase: f'{feature_name}:{phrase}', sorted_phrases))
    feature_names += f_names

CPU times: user 19.5 s, sys: 832 ms, total: 20.4 s
Wall time: 21.9 s


In [ ]:
# Fit model with transformed data, but with whole data set to see what features affect the model greatly

log_reg = LogisticRegression(random_state=42, n_jobs=-1).fit(X, y)

# Feature Importance Scores

In [ ]:
# Check feature importances

eli5.explain_weights(log_reg, feature_names=feature_names, top=(30, 30))

Weight?,Feature
+2.121,industry:Oil & Energy
+2.003,department:Information Technology
+1.879,city:San Mateo
+1.875,country:MY
+1.798,city:AUSTIN
+1.612,industry:Accounting
+1.562,title:administrative
+1.520,company_profile:accion
+1.486,department:Oil & Energy
+1.482,title:position


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   17880 non-null  object
 1   department              17880 non-null  object
 2   company_profile         17880 non-null  object
 3   description             17880 non-null  object
 4   requirements            17880 non-null  object
 5   benefits                17880 non-null  object
 6   telecommuting           17880 non-null  int64 
 7   has_company_logo        17880 non-null  int64 
 8   has_questions           17880 non-null  int64 
 9   employment_type         17880 non-null  object
 10  required_experience     17880 non-null  object
 11  required_education      17880 non-null  object
 12  industry                17880 non-null  object
 13  function                17880 non-null  object
 14  fraudulent              17880 non-null  int64 
 15  ha

# Drop certain columns and re run model

In [ ]:
### Remove some columns and test again, below codes and reasonings all the same

X2, y2 = data.drop(["fraudulent", "has_dollar_sign", "has_email", "has_phone_number", "has_employment_type",
                    'benefits_length', 'description_length'], axis=1), data.fraudulent

In [ ]:
new_text_features = text_features = ['title', 'company_profile', 'description', 'requirements', 'benefits']
new_bin_features = ['telecommuting', 'has_company_logo', 'has_questions', 'has_company_profile', 
                'has_requirements', 'has_benefits', 'has_description', 'salary_specified']
new_cat_features = ['department', 'employment_type', 'required_experience', 'required_education', 'industry', 'function', 'country', 'state', 'city']

new_num_features = ['company_profile_length', 'requirements_length', 'min_salary', 'max_salary', 'combined_text_length']

In [ ]:
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', num_transformer, new_num_features),
        ('cat', cat_transformer, new_cat_features),
        *[(feature_name, text_transformer, feature_name) 
          for feature_name in new_text_features]
    ]
)

In [ ]:
log_reg_pipe2 = Pipeline(steps=[('preprocessor2', preprocessor2),
                               ('classifier2', LogisticRegression())])

In [ ]:
%%time
cv_scores2 = cross_validate(log_reg_pipe2, X2, y2, return_train_score=True, cv=skf, 
                           scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'neg_mean_squared_error'], n_jobs=-1)

print(f'Accuracy on train part: {cv_scores2["train_accuracy"]}, mean: {cv_scores2["train_accuracy"].mean()}')
print(f'Accuracy on test part:  {cv_scores2["test_accuracy"]}, mean: {cv_scores2["test_accuracy"].mean()}')

print(f'Precision on train part: {cv_scores2["train_precision"]}, mean: {cv_scores2["train_precision"].mean()}')
print(f'Precision on test part:  {cv_scores2["test_precision"]}, mean: {cv_scores2["test_precision"].mean()}')

print(f'Recall on train part: {cv_scores2["train_recall"]}, mean: {cv_scores2["train_recall"].mean()}')
print(f'Recall on test part:  {cv_scores2["test_recall"]}, mean: {cv_scores2["test_recall"].mean()}')

print(f'F1 on train part: {cv_scores2["train_f1"]}, mean: {cv_scores2["train_f1"].mean()}')
print(f'F1 on test part:  {cv_scores2["test_f1"]}, mean: {cv_scores2["test_f1"].mean()}')

print(f'ROC AUC on train part: {cv_scores2["train_roc_auc"]}, mean: {cv_scores2["train_roc_auc"].mean()}')
print(f'ROC AUC on test part:  {cv_scores2["test_roc_auc"]}, mean: {cv_scores2["test_roc_auc"].mean()}')

print(f'mse on train part: {cv_scores2["train_neg_mean_squared_error"]}, mean: {cv_scores2["train_neg_mean_squared_error"].mean()}')
print(f'mse on test part:  {cv_scores2["test_neg_mean_squared_error"]}, mean: {cv_scores2["test_neg_mean_squared_error"].mean()}')

Accuracy on train part: [0.99552573 0.9956003  0.99492916 0.99515287], mean: 0.9953020134228188
Accuracy on test part:  [0.98568233 0.98411633 0.98389262 0.97941834], mean: 0.9832774049217001
Precision on train part: [0.99496644 0.99663866 0.9965812  0.99829352], mean: 0.9966199525886908
Precision on test part:  [0.99350649 0.97385621 0.97385621 0.94326241], mean: 0.9711203307886662
Recall on train part: [0.91230769 0.91230769 0.89830508 0.90138675], mean: 0.9060768045513808
Recall on test part:  [0.70833333 0.68981481 0.68663594 0.61290323], mean: 0.6744218296637651
F1 on train part: [0.95184591 0.95261044 0.94489465 0.94736842], mean: 0.9491798553153736
F1 on test part:  [0.82702703 0.80758808 0.80540541 0.74301676], mean: 0.7957593170224319
ROC AUC on train part: [0.99989945 0.99991862 0.99992369 0.99991475], mean: 0.9999141259759764
ROC AUC on test part:  [0.98803087 0.98436221 0.9914552  0.98912884], mean: 0.9882442837546115
mse on train part: [-0.00447427 -0.0043997  -0.00507084 

In [ ]:
%%time
new_feature_names = new_num_features.copy()

new_num_features_scaled = StandardScaler().fit_transform(data[new_num_features])
X = new_num_features_scaled

new_feature_names += new_bin_features
X = np.hstack([X, data[new_bin_features]])


for feature_name in new_cat_features:
    encoder = OneHotEncoder()
    encoded_feature = encoder.fit_transform(data[feature_name].values.reshape(-1, 1))
    
    X = sparse_hstack([X, encoded_feature])
    f_names = list(map(lambda cat: f'{feature_name}:{cat}', encoder.categories_[0]))
    new_feature_names += f_names

for feature_name in new_text_features:
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    vectorized_feature = vectorizer.fit_transform(data[feature_name])
    
    X = sparse_hstack([X, vectorized_feature])
    sorted_phrases = [pair[0] for pair in list(sorted(vectorizer.vocabulary_.items(), 
                                                      key=lambda pair: pair[1]))]
    f_names = list(map(lambda phrase: f'{feature_name}:{phrase}', sorted_phrases))
    new_feature_names += f_names

CPU times: user 18.2 s, sys: 338 ms, total: 18.5 s
Wall time: 18.5 s


In [ ]:
log_reg2 = LogisticRegression(random_state=42, n_jobs=-1).fit(X2, y2)

In [ ]:
eli5.explain_weights(log_reg2, feature_names=new_feature_names, top=(30, 30))

Weight?,Feature
+2.123,industry:Oil & Energy
+2.001,department:Information Technology
+1.881,city:San Mateo
+1.879,country:MY
+1.793,city:AUSTIN
+1.615,industry:Accounting
+1.562,title:administrative
+1.518,company_profile:accion
+1.485,department:Oil & Energy
+1.481,title:position
